In [1]:
from scipy.io import loadmat, savemat
import numpy as np
import networkx as nx

In [2]:
import zipfile

with zipfile.ZipFile('data/nonrigid3d.zip', 'r') as zip_ref:
    zip_ref.extractall('data/')

In [3]:
# Load the .mat file
shape_path = 'data/nonrigid3d/cat0.mat'
mat_data = loadmat(shape_path)
surface_data = mat_data['surface']

# Extract vertices and triangle indices
X = surface_data['X'][0, 0].flatten()
Y = surface_data['Y'][0, 0].flatten()
Z = surface_data['Z'][0, 0].flatten()
vertices = np.column_stack((X, Y, Z))
triangles = surface_data['TRIV'][0, 0] - 1  # Adjust for 0-based indexing in Python


In [4]:
vertices.shape, triangles.shape

((3400, 3), (6768, 3))

In [5]:
triangles[0,:]

array([ 8, 16,  4], dtype=uint16)

In [6]:
G = nx.Graph()
for i, vertex in enumerate(vertices):
    G.add_node(i, pos=vertex)

for triangle in triangles:
    edge1 = (triangle[0], triangle[1])
    edge2 = (triangle[0], triangle[2])
    edge3 = (triangle[1], triangle[2])

    weight_1 = np.linalg.norm(vertices[edge1[0]] - vertices[edge1[1]])
    weight_2 = np.linalg.norm(vertices[edge2[0]] - vertices[edge2[1]])
    weight_3 = np.linalg.norm(vertices[edge3[0]] - vertices[edge3[1]])

    G.add_edge(*edge1, weight=weight_1)
    G.add_edge(*edge2, weight=weight_2)
    G.add_edge(*edge3, weight=weight_3)



In [7]:
import time
st = time.time()
l = nx.shortest_path_length(G, 0,20, weight='weight')
print((time.time()-st)*G.number_of_nodes())
l

35.327768325805664


32.68497073110362

In [8]:
from tqdm.notebook import tqdm
def geodesic_distance_matrix(G):
    n = G.number_of_nodes()
    distance_matrix = np.zeros((n, n))
    tqdm_bar = tqdm(range(n*n))
    for i in range(n):
        for j in range(n):
            distance_matrix[i, j] = nx.shortest_path_length(G, i, j, weight='weight')
            tqdm_bar.update(1)
    return distance_matrix


In [9]:
import src.preprocessing.preprocessing as pp
D = pp.construct_geodesic_distance_matrix(G, weight=None, use_parallel=False)
# list = pp.farthest_first_sampling(G, 200)

Average time per calculation: 0.01011962890625 sec
Estimated time: min, 1949.7151692708333, sec, 42.91015625
Use single thread
The current weight setting is None


  0%|          | 0/3400 [00:00<?, ?it/s]

  0%|          | 0/3400 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# n = G.number_of_nodes()
# st = time.time()
# for i in range(n):
#     nx.shortest_path_length(G, 0, i, weight='weight')
# elapsed = time.time() - st
# print(f"Elapsed time: {elapsed} seconds, total estimate = {elapsed*n} seconds")

: 

: 

: 